# COSMOSWeb Master Catalog Exploration

This notebook explores the large COSMOSWeb master catalog FITS file to:
- Understand the catalog structure and available columns
- Apply initial filtering criteria efficiently
- Create a manageable subset for detailed pair selection
- Export filtered data for further analysis

**Note**: This catalog is very large (~9GB), so we'll work with chunks and apply filters early to manage memory usage.

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
import gc
import warnings
warnings.filterwarnings('ignore')

# Define cosmology
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)

## 1. Initial FITS File Inspection

In [22]:
# First, let's inspect the FITS file structure without loading all data
fits_file = '../data/raw/COSMOSWeb_mastercatalog_v1.fits'

print("=== FITS File Information ===")
with fits.open(fits_file) as hdul:
    print(f"Number of HDUs: {len(hdul)}")
    hdul.info()
    
    print("\n=== Detailed HDU Information ===")
    for i, hdu in enumerate(hdul):
        print(f"\nHDU {i}: {hdu.name if hasattr(hdu, 'name') else 'Primary'}")
        print(f"  Type: {type(hdu).__name__}")
        
        if hasattr(hdu, 'header'):
            # Print relevant header info
            print(f"  Header keys: {len(hdu.header)} entries")
            if 'EXTNAME' in hdu.header:
                print(f"  Extension name: {hdu.header['EXTNAME']}")
            if 'TTYPE1' in hdu.header:
                print(f"  First column: {hdu.header['TTYPE1']}")
        
        # Check if it's a binary table with data
        if hasattr(hdu, 'columns') and hdu.columns is not None:
            print(f"  Columns: {len(hdu.columns)}")
            print(f"  Rows: {hdu.header.get('NAXIS2', 'Unknown')}")
            
            # Show first 10 column names
            col_names = [col.name for col in hdu.columns[:10]]
            print(f"  Sample columns: {', '.join(col_names)}")
            if len(hdu.columns) > 10:
                print(f"    ... and {len(hdu.columns) - 10} more")
                
        elif hasattr(hdu, 'data') and hdu.data is not None:
            print(f"  Data shape: {hdu.data.shape}")
            print(f"  Data type: {hdu.data.dtype}")
        else:
            print("  No data or header-only HDU")

=== FITS File Information ===
Number of HDUs: 7
Filename: ../data/raw/COSMOSWeb_mastercatalog_v1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  PHOTOMETRY HOTCOLD AND SE++    1 BinTableHDU    603   784016R x 287C   [K, K, 3A, K, D, D, D, D, D, D, D, D, D, 4A, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, 5D, 5D, 5D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, L, L, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D

In [23]:
# Explore all HDUs and identify which ones contain data
print("=== Exploring All HDUs for Data ===")

data_hdus = []
with fits.open(fits_file) as hdul:
    for i, hdu in enumerate(hdul):
        if hasattr(hdu, 'columns') and hdu.columns is not None and len(hdu.columns) > 0:
            n_rows = hdu.header.get('NAXIS2', 0)
            n_cols = len(hdu.columns)
            ext_name = hdu.header.get('EXTNAME', f'HDU_{i}')
            
            data_hdus.append({
                'hdu_index': i,
                'ext_name': ext_name,
                'n_rows': n_rows,
                'n_cols': n_cols,
                'columns': [col.name for col in hdu.columns]
            })
            
            print(f"HDU {i} ({ext_name}): {n_rows:,} rows × {n_cols} columns")
            # Show first 10 column names
            col_sample = [col.name for col in hdu.columns[:10]]
            print(f"  Sample columns: {', '.join(col_sample)}")
            if len(hdu.columns) > 10:
                print(f"  ... and {len(hdu.columns) - 10} more")

print(f"\nFound {len(data_hdus)} HDUs with data tables")

# Look for common ID/matching columns across HDUs
if len(data_hdus) > 1:
    print("\n=== Checking for Common ID Columns Across HDUs ===")
    
    # Common ID column patterns
    id_patterns = ['id', 'ID', 'object', 'source', 'number', 'index']
    
    for hdu_info in data_hdus:
        print(f"HDU {hdu_info['hdu_index']} ({hdu_info['ext_name']}):")
        id_cols = []
        for pattern in id_patterns:
            id_cols.extend([col for col in hdu_info['columns'] if pattern.lower() in col.lower()])
        
        if id_cols:
            print(f"  Potential ID columns: {', '.join(set(id_cols))}")
        else:
            print("  No obvious ID columns found")
            # Show first few columns as potential IDs
            print(f"  First columns (potential IDs): {', '.join(hdu_info['columns'][:3])}")
else:
    print("Only one data HDU found - no merging needed")

=== Exploring All HDUs for Data ===
HDU 1 (PHOTOMETRY HOTCOLD AND SE++): 784,016 rows × 287 columns
  Sample columns: id, segment-id, tile, id_specz_khostovan25, ra, dec, x_image, y_image, a_image, b_image
  ... and 277 more
HDU 2 (LEPHARE): 784,016 rows × 43 columns
  Sample columns: zfinal, type, zpdf_med, zpdf_l68, zpdf_u68, zchi2, chi2_best, nbfilt, zp_agn, chi2_agn
  ... and 33 more
HDU 3 (SE++APER): 784,016 rows × 148 columns
  Sample columns: mag_aper_f115w, mag_aper_f150w, mag_aper_f277w, mag_aper_f444w, mag_aper_hst-f814w, mag_aper_f770w, mag_aper_cfht-u, mag_aper_hsc-g, mag_aper_hsc-r, mag_aper_hsc-i
  ... and 138 more
HDU 4 (CIGALE): 784,016 rows × 54 columns
  Sample columns: age_form, age_form_err, sfr_mass_vector_dir, sfr_mass_vector_dir_err, sfh_sfr_bin1, sfh_sfr_bin1_err, sfh_sfr_bin2, sfh_sfr_bin2_err, sfh_sfr_bin3, sfh_sfr_bin3_err
  ... and 44 more
HDU 5 (ML-MORPHO): 784,016 rows × 150 columns
  Sample columns: sph_0_f150w, disk_0_f150w, irr_0_f150w, bd_0_f150w, sph_

In [ ]:
# Load and combine data from all HDUs
print("=== Loading Data from All HDUs ===")

# Define the ID column to use for merging (UPDATE AFTER RUNNING PREVIOUS CELL)
MERGE_COLUMN = None  # e.g., 'id', 'ID', 'object_id', etc.

print("MANUAL UPDATE REQUIRED:")
print("After seeing the HDU analysis above, update MERGE_COLUMN with the common ID column name")
print("Example: MERGE_COLUMN = 'id'  # or 'ID', 'object_id', etc.")
print("If HDUs have different ID column names, you may need to rename them during loading\n")

def load_all_hdus(filename, merge_col=None):
    """
    Load data from all HDUs and optionally merge them
    """
    tables = {}
    
    with fits.open(filename) as hdul:
        print(f"Loading data from {len(data_hdus)} HDUs...")
        
        for hdu_info in data_hdus:
            hdu_idx = hdu_info['hdu_index']
            ext_name = hdu_info['ext_name']
            
            print(f"Loading HDU {hdu_idx} ({ext_name})...")
            try:
                # Load the HDU data
                table = Table(hdul[hdu_idx].data)
                tables[ext_name] = table
                print(f"  ✓ Loaded {len(table)} rows × {len(table.colnames)} columns")
                
            except Exception as e:
                print(f"  ✗ Error loading HDU {hdu_idx}: {e}")
                continue
    
    if len(tables) == 1:
        # Only one table, return it directly
        table_name = list(tables.keys())[0]
        print(f"\nReturning single table: {table_name}")
        return tables[table_name], {}
        
    elif len(tables) > 1 and merge_col is not None:
        # Multiple tables, try to merge them
        print(f"\n=== Merging {len(tables)} tables on column '{merge_col}' ===")
        
        table_names = list(tables.keys())
        merged_table = tables[table_names[0]]
        merge_info = {table_names[0]: len(merged_table)}
        
        print(f"Starting with {table_names[0]}: {len(merged_table)} rows")
        
        for table_name in table_names[1:]:
            table_to_merge = tables[table_name]
            
            # Check if merge column exists in both tables
            if merge_col in merged_table.colnames and merge_col in table_to_merge.colnames:
                print(f"Merging {table_name}: {len(table_to_merge)} rows")
                
                # Convert to pandas for easier merging
                df1 = merged_table.to_pandas()
                df2 = table_to_merge.to_pandas()
                
                # Merge on the common ID column
                merged_df = pd.merge(df1, df2, on=merge_col, how='inner', suffixes=('', f'_{table_name}'))
                
                # Convert back to astropy Table
                merged_table = Table.from_pandas(merged_df)
                merge_info[table_name] = len(table_to_merge)
                
                print(f"  After merge: {len(merged_table)} rows")
                
            else:
                print(f"  ✗ Cannot merge {table_name}: missing column '{merge_col}'")
                if merge_col not in table_to_merge.colnames:
                    print(f"    Available columns: {', '.join(table_to_merge.colnames[:10])}...")
        
        print(f"\nFinal merged table: {len(merged_table)} rows × {len(merged_table.colnames)} columns")
        return merged_table, merge_info
        
    else:
        # Multiple tables but no merge column specified
        print(f"\n=== Returning {len(tables)} separate tables ===")
        print("No merge column specified. Consider setting MERGE_COLUMN to combine tables.")
        return tables, {}

# Load the data
if MERGE_COLUMN is not None:
    catalog, merge_info = load_all_hdus(fits_file, MERGE_COLUMN)
    if isinstance(catalog, dict):
        print("Multiple tables loaded separately:")
        for name, table in catalog.items():
            print(f"  {name}: {len(table)} rows × {len(table.colnames)} columns")
        # For now, use the first table for analysis
        catalog = list(catalog.values())[0]
        print(f"Using first table for analysis: {len(catalog)} rows")
else:
    print("Please set MERGE_COLUMN first!")

In [24]:
# Alternative: Load all HDUs without merging (safer approach)
print("=== Alternative: Loading All HDUs Separately ===")

def load_all_hdus_separate(filename):
    """
    Load all HDUs as separate tables for inspection
    """
    all_tables = {}
    
    with fits.open(filename) as hdul:
        print(f"Loading all {len(data_hdus)} data HDUs...")
        
        for hdu_info in data_hdus:
            hdu_idx = hdu_info['hdu_index']
            ext_name = hdu_info['ext_name']
            
            try:
                print(f"Loading HDU {hdu_idx} ({ext_name})...")
                table = Table(hdul[hdu_idx].data)
                all_tables[ext_name] = table
                print(f"  ✓ {len(table):,} rows × {len(table.colnames)} columns")
                
                # Memory usage estimate
                memory_mb = len(table) * len(table.colnames) * 8 / 1e6
                print(f"  Memory: ~{memory_mb:.1f} MB")
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
    
    return all_tables

# Load all tables
all_catalogs = load_all_hdus_separate(fits_file)

print(f"\n=== Loaded {len(all_catalogs)} catalogs ===")
total_memory = 0
for name, table in all_catalogs.items():
    memory_mb = len(table) * len(table.colnames) * 8 / 1e6
    total_memory += memory_mb
    print(f"{name}: {len(table):,} objects, {len(table.colnames)} columns")

print(f"\nTotal estimated memory: ~{total_memory:.1f} MB")

# Choose primary catalog for analysis
if len(all_catalogs) > 0:
    # Use the largest table as primary, or specify manually
    primary_name = max(all_catalogs.keys(), key=lambda k: len(all_catalogs[k]))
    catalog = all_catalogs[primary_name]
    print(f"\nUsing '{primary_name}' as primary catalog ({len(catalog):,} objects)")
    
    # Store others for later use if needed
    secondary_catalogs = {k: v for k, v in all_catalogs.items() if k != primary_name}
    if secondary_catalogs:
        print(f"Secondary catalogs available: {', '.join(secondary_catalogs.keys())}")
else:
    print("No catalogs loaded successfully!")

=== Alternative: Loading All HDUs Separately ===
Loading all 6 data HDUs...
Loading HDU 1 (PHOTOMETRY HOTCOLD AND SE++)...
  ✓ 784,016 rows × 287 columns
  Memory: ~1800.1 MB
Loading HDU 2 (LEPHARE)...
  ✓ 784,016 rows × 43 columns
  Memory: ~269.7 MB
Loading HDU 3 (SE++APER)...
  ✓ 784,016 rows × 148 columns
  Memory: ~928.3 MB
Loading HDU 4 (CIGALE)...
  ✓ 784,016 rows × 54 columns
  Memory: ~338.7 MB
Loading HDU 5 (ML-MORPHO)...
  ✓ 784,016 rows × 150 columns
  Memory: ~940.8 MB
Loading HDU 6 (B+D)...
  ✓ 784,016 rows × 461 columns
  Memory: ~2891.5 MB

=== Loaded 6 catalogs ===
PHOTOMETRY HOTCOLD AND SE++: 784,016 objects, 287 columns
LEPHARE: 784,016 objects, 43 columns
SE++APER: 784,016 objects, 148 columns
CIGALE: 784,016 objects, 54 columns
ML-MORPHO: 784,016 objects, 150 columns
B+D: 784,016 objects, 461 columns

Total estimated memory: ~7169.0 MB

Using 'PHOTOMETRY HOTCOLD AND SE++' as primary catalog (784,016 objects)
Secondary catalogs available: LEPHARE, SE++APER, CIGALE, 

In [25]:
# Concatenate all catalogs horizontally (side by side)
print("=== CONCATENATING ALL CATALOGS HORIZONTALLY ===")

def concatenate_all_catalogs(all_cats):
    """
    Concatenate all catalogs horizontally, assuming rows are in the same order
    """
    if not all_cats:
        print("No catalogs to concatenate!")
        return None
    
    print(f"Concatenating {len(all_cats)} catalogs...")
    
    # Check that all catalogs have the same number of rows
    row_counts = {name: len(table) for name, table in all_cats.items()}
    print("Row counts per catalog:")
    for name, count in row_counts.items():
        print(f"  {name}: {count:,} rows")
    
    # Check if all have the same number of rows
    unique_counts = set(row_counts.values())
    if len(unique_counts) > 1:
        print(f"⚠ WARNING: Catalogs have different numbers of rows: {unique_counts}")
        print("Using the smallest common size...")
        min_rows = min(row_counts.values())
        print(f"Will use first {min_rows:,} rows from each catalog")
    else:
        min_rows = list(unique_counts)[0]
        print(f"✓ All catalogs have {min_rows:,} rows")
    
    # Start with the first catalog
    cat_names = list(all_cats.keys())
    combined_table = all_cats[cat_names[0]][:min_rows].copy()
    
    print(f"Starting with {cat_names[0]}: {len(combined_table.colnames)} columns")
    
    # Add columns from other catalogs
    for cat_name in cat_names[1:]:
        source_table = all_cats[cat_name][:min_rows]
        
        print(f"Adding {cat_name}: {len(source_table.colnames)} columns")
        
        # Check for column name conflicts
        conflicts = set(combined_table.colnames) & set(source_table.colnames)
        if conflicts:
            print(f"  Column conflicts: {', '.join(list(conflicts)[:5])}...")
            # Rename conflicting columns with suffix
            for col in source_table.colnames:
                if col in combined_table.colnames:
                    new_col_name = f"{col}_{cat_name}"
                    source_table.rename_column(col, new_col_name)
                    print(f"    Renamed {col} → {new_col_name}")
        
        # Add all columns from this catalog
        for col_name in source_table.colnames:
            combined_table[col_name] = source_table[col_name]
    
    print(f"\n✓ Combined catalog created:")
    print(f"  Rows: {len(combined_table):,}")
    print(f"  Columns: {len(combined_table.colnames)}")
    print(f"  Estimated memory: ~{len(combined_table) * len(combined_table.colnames) * 8 / 1e6:.1f} MB")
    
    return combined_table

# Concatenate all catalogs
if 'all_catalogs' in locals() and all_catalogs:
    unified_catalog = concatenate_all_catalogs(all_catalogs)
    
    if unified_catalog is not None:
        print(f"\n=== SUCCESS ===")
        print(f"Unified catalog: {len(unified_catalog):,} objects × {len(unified_catalog.colnames)} columns")
        
        # Set this as the main catalog for analysis
        catalog = unified_catalog
        print("✓ Set as main 'catalog' variable for further analysis")
    else:
        print("Failed to create unified catalog")
else:
    print("ERROR: all_catalogs not found. Please run the previous cells first.")

=== CONCATENATING ALL CATALOGS HORIZONTALLY ===
Concatenating 6 catalogs...
Row counts per catalog:
  PHOTOMETRY HOTCOLD AND SE++: 784,016 rows
  LEPHARE: 784,016 rows
  SE++APER: 784,016 rows
  CIGALE: 784,016 rows
  ML-MORPHO: 784,016 rows
  B+D: 784,016 rows
✓ All catalogs have 784,016 rows
Starting with PHOTOMETRY HOTCOLD AND SE++: 287 columns
Adding LEPHARE: 43 columns
Adding SE++APER: 148 columns
  Column conflicts: flux_err_aper_f150w, mag_aper_f770w, flux_aper_hst-f814w, mag_aper_f115w, flux_err_aper_f277w...
    Renamed mag_aper_f115w → mag_aper_f115w_SE++APER
    Renamed mag_aper_f150w → mag_aper_f150w_SE++APER
    Renamed mag_aper_f277w → mag_aper_f277w_SE++APER
    Renamed mag_aper_f444w → mag_aper_f444w_SE++APER
    Renamed mag_aper_hst-f814w → mag_aper_hst-f814w_SE++APER
    Renamed mag_aper_f770w → mag_aper_f770w_SE++APER
    Renamed flux_aper_f115w → flux_aper_f115w_SE++APER
    Renamed flux_aper_f150w → flux_aper_f150w_SE++APER
    Renamed flux_aper_f277w → flux_aper_f

In [26]:
# Comprehensive column analysis across all loaded catalogs
print("=== COMPREHENSIVE COLUMN ANALYSIS ===")

if 'all_catalogs' in locals() and all_catalogs:
    all_columns = {}
    
    # Collect all column names from all catalogs
    for cat_name, table in all_catalogs.items():
        all_columns[cat_name] = table.colnames
        print(f"\n{cat_name}: {len(table.colnames)} columns")
        
        # Print columns in organized format
        cols_per_row = 4
        for i in range(0, len(table.colnames), cols_per_row):
            row_cols = table.colnames[i:i+cols_per_row]
            formatted_cols = [f"{j+i+1:3d}. {col:<20}" for j, col in enumerate(row_cols)]
            print("    " + " ".join(formatted_cols))
    
    # Create comprehensive column search across ALL catalogs
    print("\n" + "="*100)
    print("=== SEARCHING ALL CATALOGS FOR KEY COLUMNS ===")
    
    key_patterns = {
        'id': ['id', 'ID', 'object', 'source', 'number', 'index'],
        'coordinates': ['RA', 'DEC', 'ra', 'dec', 'ALPHA', 'DELTA', 'alpha', 'delta', 'coord'],
        'redshift': ['z', 'Z', 'redshift', 'REDSHIFT', 'z_phot', 'z_spec', 'zphot', 'zspec', 'zbest'],
        'stellar_mass': ['mass', 'MASS', 'lmass', 'stellar', 'logm', 'mstar', 'M_star'],
        'sfr': ['sfr', 'SFR', 'star_formation', 'formation_rate', 'psi'],
        'size': ['re', 'RE', 'r_eff', 'radius', 'size', 'kron', 'petro', 'half_light'],
        'magnitude': ['mag', 'MAG', 'flux', 'FLUX', 'f606w', 'f814w', 'f160w', 'AB'],
        'colors': ['color', 'COLOR', 'g_r', 'r_i', 'i_z'],
        'morphology': ['morph', 'class', 'type', 'flag', 'stellar', 'galaxy', 'sersic'],
        'photometry': ['f435w', 'f606w', 'f814w', 'f125w', 'f140w', 'f160w', 'ks', 'irac', 'wise'],
        'environment': ['density', 'neighbor', 'isolation', 'group'],
        'quality': ['flag', 'qual', 'error', 'chi', 'prob']
    }
    
    # Search for patterns across all catalogs
    global_found_cols = {}
    for category, patterns in key_patterns.items():
        category_matches = {}
        
        for cat_name, columns in all_columns.items():
            matches = []
            for pattern in patterns:
                matches.extend([col for col in columns if pattern.lower() in col.lower()])
            
            if matches:
                category_matches[cat_name] = list(set(matches))
        
        if category_matches:
            global_found_cols[category] = category_matches
            
            print(f"\n{category.upper()}:")
            for cat_name, cols in category_matches.items():
                print(f"  {cat_name}: {', '.join(cols[:5])}")
                if len(cols) > 5:
                    print(f"    ... and {len(cols) - 5} more")
    
    # Create unified column mapping suggestions
    print("\n" + "="*100)
    print("=== SUGGESTED COLUMN MAPPING ===")
    
    suggested_mapping = {}
    for category, cat_matches in global_found_cols.items():
        if cat_matches:
            # Find the catalog with the most matches for this category
            best_cat = max(cat_matches.keys(), key=lambda k: len(cat_matches[k]))
            best_cols = cat_matches[best_cat]
            
            # Suggest the first/best column
            suggested_mapping[category] = {
                'catalog': best_cat,
                'column': best_cols[0],
                'alternatives': best_cols[1:5] if len(best_cols) > 1 else []
            }
    
    print("Recommended columns for analysis:")
    print("column_mapping = {")
    for category, info in suggested_mapping.items():
        alternatives_str = f" # alternatives: {', '.join(info['alternatives'])}" if info['alternatives'] else ""
        print(f"    '{category}': ('{info['catalog']}', '{info['column']}'),{alternatives_str}")
    print("}")
    
else:
    print("No catalogs loaded. Please run the previous cells first.")

=== COMPREHENSIVE COLUMN ANALYSIS ===

PHOTOMETRY HOTCOLD AND SE++: 287 columns
      1. id                     2. segment-id             3. tile                   4. id_specz_khostovan25
      5. ra                     6. dec                    7. x_image                8. y_image             
      9. a_image               10. b_image               11. theta_image           12. theta_world         
     13. chi2_max              14. mode                  15. snr_hst-f814w         16. wht_hst-f814w       
     17. flux_auto_hst-f814w   18. flux_err_auto_hst-f814w  19. mag_auto_hst-f814w    20. flux_aper_hst-f814w 
     21. flux_err_aper_hst-f814w  22. mag_aper_hst-f814w    23. snr_f115w             24. wht_f115w           
     25. flux_auto_f115w       26. flux_err_auto_f115w   27. mag_auto_f115w        28. flux_aper_f115w     
     29. flux_err_aper_f115w   30. mag_aper_f115w        31. snr_f150w             32. wht_f150w           
     33. flux_auto_f150w       34. flux_err_auto_f

## 3. Identify Key Columns for Selection

In [27]:
# Display all column names to identify the ones we need
print("=== All Available Columns ===")
print(f"Total columns: {len(catalog.colnames)}\n")

# Print columns in groups for readability
cols_per_row = 4
for i in range(0, len(catalog.colnames), cols_per_row):
    row_cols = catalog.colnames[i:i+cols_per_row]
    formatted_cols = [f"{j+i+1:3d}. {col:<20}" for j, col in enumerate(row_cols)]
    print(" ".join(formatted_cols))

print("\n" + "="*80)

# Look for key columns we need (with common naming patterns)
key_patterns = {
    'coordinates': ['RA', 'DEC', 'ra', 'dec', 'ALPHA', 'DELTA'],
    'redshift': ['z', 'Z', 'redshift', 'REDSHIFT', 'z_phot', 'z_spec'],
    'stellar_mass': ['logm', 'mass', 'MASS', 'lmass', 'stellar_mass'],
    'sfr': ['sfr', 'SFR', 'star_formation_rate'],
    'size': ['re', 'RE', 'r_eff', 'radius', 'size'],
    'morphology': ['morph', 'class', 'type', 'flag']
}

print("\n=== Searching for Key Columns ===")
found_cols = {}

for category, patterns in key_patterns.items():
    matches = []
    for pattern in patterns:
        matches.extend([col for col in catalog.colnames if pattern.lower() in col.lower()])
    
    if matches:
        found_cols[category] = list(set(matches))  # Remove duplicates
        print(f"{category.upper():<15}: {', '.join(found_cols[category])}")
    else:
        print(f"{category.upper():<15}: None found")

=== All Available Columns ===
Total columns: 1143

  1. id                     2. segment-id             3. tile                   4. id_specz_khostovan25
  5. ra                     6. dec                    7. x_image                8. y_image             
  9. a_image               10. b_image               11. theta_image           12. theta_world         
 13. chi2_max              14. mode                  15. snr_hst-f814w         16. wht_hst-f814w       
 17. flux_auto_hst-f814w   18. flux_err_auto_hst-f814w  19. mag_auto_hst-f814w    20. flux_aper_hst-f814w 
 21. flux_err_aper_hst-f814w  22. mag_aper_hst-f814w    23. snr_f115w             24. wht_f115w           
 25. flux_auto_f115w       26. flux_err_auto_f115w   27. mag_auto_f115w        28. flux_aper_f115w     
 29. flux_err_aper_f115w   30. mag_aper_f115w        31. snr_f150w             32. wht_f150w           
 33. flux_auto_f150w       34. flux_err_auto_f150w   35. mag_auto_f150w        36. flux_aper_f150w     
 37. fl

In [29]:
# Define column mapping across multiple catalogs
# UPDATE THESE BASED ON THE OUTPUT OF THE PREVIOUS CELL
column_mapping = {
    # Format: 'parameter': ('catalog_name', 'column_name')
    'id': ('PHOTOMETRY HOTCOLD AND SE++', 'id'),
    'ra': ('PHOTOMETRY HOTCOLD AND SE++', 'ra'),
    'dec': ('PHOTOMETRY HOTCOLD AND SE++', 'dec'),  
    'redshift': ('LEPHARE', 'zfinal'),
    'stellar_mass': ('CIGALE', 'mass'),
    'sfr': ('CIGALE', 'sfr_inst'),
    'size': ('PHOTOMETRY HOTCOLD AND SE++', 'radius_sersic'),  # Effective radius of the Sersic model [deg]
}

print("MANUAL UPDATE REQUIRED:")
print("Update the column_mapping dictionary above based on the previous cell output.")
print("Format: 'parameter': ('catalog_name', 'column_name')")
print("\\nExample:")
print("column_mapping = {")
print("    'id': ('HDU_1', 'ID'),")
print("    'ra': ('HDU_1', 'RA'),")
print("    'dec': ('HDU_1', 'DEC'),") 
print("    'redshift': ('photometry_cat', 'z_phot'),")
print("    'stellar_mass': ('derived_props', 'logmass'),")
print("    'sfr': ('derived_props', 'SFR'),")
print("    'size': ('morphology', 're_arcsec'),")
print("}")

def create_unified_catalog(all_cats, col_map):
    """
    Create a unified catalog by extracting specified columns from different catalogs
    """
    print("=== Creating Unified Catalog ===")
    
    # Check if all required catalogs and columns exist
    missing_items = []
    for param, (cat_name, col_name) in col_map.items():
        if cat_name is None or col_name is None:
            missing_items.append(f"{param}: ({cat_name}, {col_name})")
            continue
            
        if cat_name not in all_cats:
            missing_items.append(f"Catalog '{cat_name}' not found for {param}")
            continue
            
        if col_name not in all_cats[cat_name].colnames:
            missing_items.append(f"Column '{col_name}' not found in {cat_name} for {param}")
            continue
    
    if missing_items:
        print("ERROR: Missing items in column mapping:")
        for item in missing_items:
            print(f"  - {item}")
        print("Please update column_mapping first!")
        return None
    
    # Get the primary catalog (usually contains ID and coordinates)
    primary_info = col_map['id']
    primary_cat_name = primary_info[0]
    primary_table = all_cats[primary_cat_name]
    id_col = primary_info[1]
    
    print(f"Using '{primary_cat_name}' as primary catalog ({len(primary_table)} objects)")
    
    # Start with the primary catalog's ID column
    unified_data = {
        'id': primary_table[id_col],
    }
    
    # Extract columns from each catalog
    for param, (cat_name, col_name) in col_map.items():
        if param == 'id':
            continue  # Already added
            
        source_table = all_cats[cat_name]
        print(f"Adding {param} from {cat_name}.{col_name}")
        
        if cat_name == primary_cat_name:
            # Same catalog, direct assignment
            unified_data[param] = source_table[col_name]
        else:
            # Different catalog, need to match by ID
            # This assumes all catalogs have the same ID column
            source_id_col = col_map['id'][1]  # Use same ID column name
            
            if source_id_col in source_table.colnames:
                # Convert to pandas for easier merging
                primary_df = pd.DataFrame({'id': unified_data['id']})
                source_df = source_table[[source_id_col, col_name]].to_pandas()
                source_df = source_df.rename(columns={source_id_col: 'id'})
                
                # Merge
                merged = pd.merge(primary_df, source_df, on='id', how='left')
                unified_data[param] = merged[col_name].values
                
                n_matched = (~pd.isna(merged[col_name])).sum()
                print(f"  Matched {n_matched}/{len(primary_table)} objects ({n_matched/len(primary_table)*100:.1f}%)")
            else:
                print(f"  WARNING: ID column '{source_id_col}' not found in {cat_name}")
                # Fill with NaN
                unified_data[param] = np.full(len(primary_table), np.nan)
    
    # Create unified astropy Table
    unified_table = Table(unified_data)
    print(f"\\nUnified catalog created: {len(unified_table)} objects × {len(unified_table.colnames)} columns")
    
    return unified_table

# Create unified catalog (only if column_mapping is properly defined)
if all(v[0] is not None and v[1] is not None for v in column_mapping.values()):
    catalog = create_unified_catalog(all_catalogs, column_mapping)
else:
    print("Please update column_mapping first!")
    print("Available catalogs:", list(all_catalogs.keys()) if 'all_catalogs' in locals() else "None")

MANUAL UPDATE REQUIRED:
Update the column_mapping dictionary above based on the previous cell output.
Format: 'parameter': ('catalog_name', 'column_name')
\nExample:
column_mapping = {
    'id': ('HDU_1', 'ID'),
    'ra': ('HDU_1', 'RA'),
    'dec': ('HDU_1', 'DEC'),
    'redshift': ('photometry_cat', 'z_phot'),
    'stellar_mass': ('derived_props', 'logmass'),
    'sfr': ('derived_props', 'SFR'),
    'size': ('morphology', 're_arcsec'),
}
=== Creating Unified Catalog ===
Using 'PHOTOMETRY HOTCOLD AND SE++' as primary catalog (784016 objects)
Adding ra from PHOTOMETRY HOTCOLD AND SE++.ra
Adding dec from PHOTOMETRY HOTCOLD AND SE++.dec
Adding redshift from LEPHARE.zfinal
Adding stellar_mass from CIGALE.mass
Adding sfr from CIGALE.sfr_inst
Adding size from PHOTOMETRY HOTCOLD AND SE++.radius_sersic
\nUnified catalog created: 784016 objects × 7 columns


In [30]:
# Let's examine the first few rows and basic statistics
# Convert to pandas for easier manipulation (if not too large)
if len(catalog) < 1000000:  # Only convert if less than 1M objects
    df = catalog.to_pandas()
    print("Converted to pandas DataFrame")
    
    # Show basic info
    print(f"\n=== Basic Information ===")
    print(f"Shape: {df.shape}")
    print(f"Data types summary:")
    print(df.dtypes.value_counts())
    
    # Show first few rows of key columns if we found them
    display_cols = []
    for category, cols in found_cols.items():
        if cols:
            display_cols.extend(cols[:2])  # Take first 2 from each category
    
    if display_cols:
        print(f"\n=== Sample Data (first 5 rows) ===")
        available_cols = [col for col in display_cols[:10] if col in df.columns]
        if available_cols:
            print(df[available_cols].head())
    
else:
    print(f"Catalog too large ({len(catalog)} objects) - keeping as astropy Table")
    print("First 3 rows:")
    print(catalog[:3])

Converted to pandas DataFrame

=== Basic Information ===
Shape: (784016, 7)
Data types summary:
float64    6
int64      1
Name: count, dtype: int64

=== Sample Data (first 5 rows) ===


In [40]:
print(df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784016 entries, 0 to 784015
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            784016 non-null  int64  
 1   ra            784016 non-null  float64
 2   dec           784016 non-null  float64
 3   redshift      0 non-null       float64
 4   stellar_mass  0 non-null       float64
 5   sfr           0 non-null       float64
 6   size          784016 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 41.9 MB
None


## 5. Define Selection Criteria and Apply Initial Filters

In [34]:
# Based on the column inspection above, define the actual column names to use
# UPDATE THESE BASED ON THE OUTPUT OF THE PREVIOUS CELLS
column_mapping = {
    'ra': 'ra',           # Replace with actual RA column name
    'dec': 'dec',          # Replace with actual DEC column name  
    'redshift': 'redshift',     # Replace with actual redshift column name
    'stellar_mass': 'stellar_mass', # Replace with actual stellar mass column name
    'sfr': 'sfr',          # Replace with actual SFR column name
    'size': 'size',         # Replace with actual size column name
}

print("MANUAL UPDATE REQUIRED:")
print("Please update the column_mapping dictionary above with the actual column names")
print("found in the previous cell output, then run the following cells.")
print("\nExample:")
print("column_mapping = {")
print("    'ra': 'RA_1',")
print("    'dec': 'DEC_1',")
print("    'redshift': 'z_phot',")
print("    'stellar_mass': 'logmass',")
print("    'sfr': 'SFR_best',")
print("    'size': 're_arcsec',")
print("}")

MANUAL UPDATE REQUIRED:
Please update the column_mapping dictionary above with the actual column names
found in the previous cell output, then run the following cells.

Example:
column_mapping = {
    'ra': 'RA_1',
    'dec': 'DEC_1',
    'redshift': 'z_phot',
    'stellar_mass': 'logmass',
    'sfr': 'SFR_best',
    'size': 're_arcsec',
}


In [44]:
 # Get valid redshift data
 REDSHIFT_COLUMN = 'redshift'
z_data = catalog[REDSHIFT_COLUMN]
valid_z = z_data[~np.isnan(z_data)]

print(f"Plotting {len(valid_z):,} objects with valid redshift data")


Plotting 0 objects with valid redshift data


In [ ]:

  # Create histogram
  plt.hist(valid_z, bins=50, alpha=0.7, edgecolor='black',
  color='skyblue')
  plt.xlabel('Redshift')
  plt.ylabel('Number of galaxies')
  plt.title(f'Redshift Distribution 
  ({REDSHIFT_COLUMN})\n{len(valid_z):,} galaxies')

  # Add statistics
  median_z = np.median(valid_z)
  plt.axvline(median_z, color='red', linestyle='--',
  linewidth=2,
             label=f'Median: {median_z:.2f}')

  # Add Halpha observable range
  z_min_halpha = 0.48
  z_max_halpha = 7.0
  plt.axvspan(z_min_halpha, z_max_halpha, alpha=0.2,
  color='green',
             label=f'Hα observable 
  ({z_min_halpha}-{z_max_halpha})')

  plt.legend()
  plt.grid(True, alpha=0.3)
  plt.tight_layout()
  plt.show()

  # Print statistics
  print(f"\n=== REDSHIFT STATISTICS ===")
  print(f"Total objects: {len(z_data):,}")
  print(f"Valid redshifts: {len(valid_z):,} 
  ({len(valid_z)/len(z_data)*100:.1f}%)")
  print(f"Range: {valid_z.min():.3f} - {valid_z.max():.3f}")
  print(f"Median: {median_z:.3f}")
  print(f"Objects in Hα range: {np.sum((valid_z >= z_min_halpha)
   & (valid_z <= z_max_halpha)):,}")


In [35]:
# Apply initial selection criteria
# Only run this cell after updating column_mapping above!

def apply_initial_selection(cat, col_map):
    """
    Apply initial selection criteria to reduce catalog size
    """
    print("=== Applying Initial Selection Criteria ===")
    print(f"Starting with: {len(cat):,} objects")
    
    # Check if we have the required columns
    missing_cols = [k for k, v in col_map.items() if v is None or v not in cat.colnames]
    if missing_cols:
        print(f"ERROR: Missing required columns: {missing_cols}")
        print("Please update column_mapping in the previous cell")
        return cat
    
    # Create boolean mask for selection
    mask = np.ones(len(cat), dtype=bool)
    
    # 1. Valid redshift range for Halpha observability (0.48 < z < 7.0)
    z_col = col_map['redshift']
    z_min, z_max = 0.48, 7.0
    z_mask = (cat[z_col] > z_min) & (cat[z_col] < z_max) & np.isfinite(cat[z_col])
    mask &= z_mask
    print(f"After redshift cut ({z_min} < z < {z_max}): {mask.sum():,} objects")
    
    # 2. Valid stellar mass range (reasonable for galaxies)
    if col_map['stellar_mass'] in cat.colnames:
        mass_col = col_map['stellar_mass']
        mass_mask = (cat[mass_col] > 8.0) & (cat[mass_col] < 12.0) & np.isfinite(cat[mass_col])
        mask &= mass_mask
        print(f"After stellar mass cut (8 < log M* < 12): {mask.sum():,} objects")
    
    # 3. Valid SFR (remove non-detections/extreme values)
    if col_map['sfr'] in cat.colnames:
        sfr_col = col_map['sfr']
        sfr_mask = (cat[sfr_col] > 0) & (cat[sfr_col] < 1000) & np.isfinite(cat[sfr_col])
        mask &= sfr_mask
        print(f"After SFR cut (0 < SFR < 1000 M☉/yr): {mask.sum():,} objects")
    
    # 4. Valid coordinates
    ra_col, dec_col = col_map['ra'], col_map['dec']
    coord_mask = np.isfinite(cat[ra_col]) & np.isfinite(cat[dec_col])
    mask &= coord_mask
    print(f"After coordinate validity check: {mask.sum():,} objects")
    
    # Apply mask
    filtered_cat = cat[mask]
    print(f"\nFinal filtered catalog: {len(filtered_cat):,} objects")
    print(f"Reduction factor: {len(cat) / len(filtered_cat):.1f}x smaller")
    
    return filtered_cat

# Apply the selection (only if column_mapping is properly defined)
if all(v is not None for v in column_mapping.values()):
    filtered_catalog = apply_initial_selection(catalog, column_mapping)
else:
    print("Please update column_mapping first!")

=== Applying Initial Selection Criteria ===
Starting with: 784,016 objects
After redshift cut (0.48 < z < 7.0): 0 objects
After stellar mass cut (8 < log M* < 12): 0 objects
After SFR cut (0 < SFR < 1000 M☉/yr): 0 objects
After coordinate validity check: 0 objects

Final filtered catalog: 0 objects


ZeroDivisionError: division by zero

## 6. Basic Visualizations of Filtered Sample

In [ ]:
# Create basic plots if filtering was successful
if 'filtered_catalog' in locals() and len(filtered_catalog) > 0:
    # Convert to pandas for plotting
    plot_df = filtered_catalog.to_pandas()
    
    # Set up plotting
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    # Get column names
    ra_col = column_mapping['ra']
    dec_col = column_mapping['dec']
    z_col = column_mapping['redshift']
    mass_col = column_mapping['stellar_mass']
    
    # 1. Sky distribution
    axes[0].scatter(plot_df[ra_col], plot_df[dec_col], s=0.5, alpha=0.5)
    axes[0].set_xlabel('RA (deg)')
    axes[0].set_ylabel('DEC (deg)')
    axes[0].set_title(f'Sky Distribution ({len(plot_df):,} objects)')
    
    # 2. Redshift distribution
    axes[1].hist(plot_df[z_col], bins=50, alpha=0.7, edgecolor='black')
    axes[1].set_xlabel('Redshift')
    axes[1].set_ylabel('Number of objects')
    axes[1].set_title('Redshift Distribution')
    axes[1].axvline(plot_df[z_col].median(), color='red', linestyle='--', 
                   label=f'Median: {plot_df[z_col].median():.2f}')
    axes[1].legend()
    
    # 3. Stellar mass distribution
    if mass_col in plot_df.columns:
        axes[2].hist(plot_df[mass_col], bins=50, alpha=0.7, edgecolor='black')
        axes[2].set_xlabel('log(M*/M☉)')
        axes[2].set_ylabel('Number of objects')
        axes[2].set_title('Stellar Mass Distribution')
        axes[2].axvline(plot_df[mass_col].median(), color='red', linestyle='--',
                       label=f'Median: {plot_df[mass_col].median():.2f}')
        axes[2].legend()
    
    # 4. Mass vs redshift
    if mass_col in plot_df.columns:
        axes[3].scatter(plot_df[z_col], plot_df[mass_col], s=0.5, alpha=0.3)
        axes[3].set_xlabel('Redshift')
        axes[3].set_ylabel('log(M*/M☉)')
        axes[3].set_title('Stellar Mass vs Redshift')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n=== Filtered Sample Statistics ===")
    print(f"Sample size: {len(plot_df):,} objects")
    print(f"Redshift range: {plot_df[z_col].min():.3f} - {plot_df[z_col].max():.3f}")
    if mass_col in plot_df.columns:
        print(f"Mass range: {plot_df[mass_col].min():.2f} - {plot_df[mass_col].max():.2f} dex")
else:
    print("No filtered catalog available for plotting. Please complete the previous steps.")

## 7. Export Filtered Catalog

In [ ]:
# Export the filtered catalog for further analysis
if 'filtered_catalog' in locals() and len(filtered_catalog) > 0:
    # Select only the columns we need to save space
    essential_cols = list(column_mapping.values())
    
    # Add any other useful columns (morphology, photometry, etc.)
    additional_cols = []
    for category, cols in found_cols.items():
        if category in ['morphology'] and cols:
            additional_cols.extend(cols[:3])  # Take first 3 morphology columns
    
    # Combine all columns we want to keep
    cols_to_keep = essential_cols + [col for col in additional_cols if col in filtered_catalog.colnames]
    cols_to_keep = list(set(cols_to_keep))  # Remove duplicates
    
    print(f"Exporting {len(cols_to_keep)} columns: {cols_to_keep}")
    
    # Create export dataframe
    export_df = filtered_catalog[cols_to_keep].to_pandas()
    
    # Rename columns to standard names
    rename_dict = {v: k for k, v in column_mapping.items() if v is not None}
    export_df = export_df.rename(columns=rename_dict)
    
    # Save to CSV
    output_file = '../data/processed/cosmos_web_filtered_catalog.csv'
    export_df.to_csv(output_file, index=False)
    
    print(f"\n=== Export Complete ===")
    print(f"Saved {len(export_df):,} objects to: {output_file}")
    print(f"File size: ~{len(export_df) * len(export_df.columns) * 8 / 1e6:.1f} MB")
    
    # Show sample of exported data
    print(f"\n=== Sample of Exported Data ===")
    print(export_df.head())
    
    # Create a summary file
    summary_file = '../data/processed/filtering_summary.txt'
    with open(summary_file, 'w') as f:
        f.write("COSMOSWeb Catalog Filtering Summary\n")
        f.write("=" * 40 + "\n\n")
        f.write(f"Original catalog size: {len(catalog):,} objects\n")
        f.write(f"Filtered catalog size: {len(export_df):,} objects\n")
        f.write(f"Reduction factor: {len(catalog) / len(export_df):.1f}x\n\n")
        f.write("Applied filters:\n")
        f.write("- Redshift: 0.48 < z < 7.0 (Halpha observable)\n")
        f.write("- Stellar mass: 8 < log(M*/M☉) < 12\n")
        f.write("- SFR: 0 < SFR < 1000 M☉/yr\n")
        f.write("- Valid coordinates\n\n")
        f.write(f"Column mapping used:\n")
        for k, v in column_mapping.items():
            f.write(f"- {k}: {v}\n")
    
    print(f"Summary saved to: {summary_file}")
    
else:
    print("No filtered catalog to export. Please complete the previous steps.")